# Phase 0v2: 데이터 전처리 + 노이즈 제거 강화

## 📌 목적
기존 Phase 0의 전처리(CSV 로드, 날짜 필터링, 중복 제거)를 수행한 뒤,
**5가지 노이즈 필터링 규칙**을 적용하여 학습 데이터 품질을 향상시킵니다.

## 📂 출력 파일
```
outputs/
├── 인기글_clean_v2.parquet          # 노이즈 제거된 인기글
├── noise_removal_report.json        # 필터링 통계 리포트
├── 익게2_for_topic.parquet          # (기존과 동일)
├── 자유게시판_for_topic.parquet
├── 연애상담소_for_topic.parquet
└── 익게1_for_topic.parquet
```

## 🔗 다음 단계
이 노트북 완료 후 → Phase 1(기존 토픽 모델링) → `phase2_dataset_v2.ipynb`

---
## 1. Google Drive 마운트

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

print("✅ Google Drive 마운트 완료")

---
## 2. 경로 설정 및 패키지

In [ ]:
%pip install -q pandas numpy pyarrow

print("✅ 기본 패키지 설치 완료")

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import re
import json
from datetime import datetime

# ── 프로젝트 경로 ──────────────────────────────────────────
PROJECT_ROOT = Path("/content/drive/MyDrive/board_crawling")
DATA_DIR     = PROJECT_ROOT / "data"
OUTPUT_DIR   = PROJECT_ROOT / "outputs"
MODEL_DIR    = PROJECT_ROOT / "models"

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

print(f"📁 PROJECT_ROOT : {PROJECT_ROOT}")
print(f"📁 DATA_DIR     : {DATA_DIR}")
print(f"📁 OUTPUT_DIR   : {OUTPUT_DIR}")

---
## 3. 유틸 함수 정의

In [ ]:
def extract_id_from_url(u: str):
    """URL에서 6자리 이상 숫자를 추출하여 post id로 활용"""
    if pd.isna(u):
        return np.nan
    m = re.search(r"(\d{6,})", str(u))
    return int(m.group(1)) if m else np.nan


STD_COLS_BASE = [
    "id", "title", "url", "date",
    "likes", "views",
    "contents", "comments", "comments_num",
]


def load_csv(path: Path, has_board: bool = False) -> pd.DataFrame:
    """CSV를 표준 스키마로 로드 + 기본 정제"""
    print(f"\n📝 Loading: {path.name}")
    df = pd.read_csv(path)

    cols = STD_COLS_BASE.copy()
    if has_board:
        cols.insert(4, "board")

    for c in cols:
        if c not in df.columns:
            df[c] = np.nan

    df = df[cols].copy()

    df["title"]    = df["title"].astype(str).fillna("")
    df["url"]      = df["url"].astype(str).fillna("")
    df["contents"] = df["contents"].astype(str).fillna("")
    df["comments"] = df["comments"].astype(str).fillna("")

    if has_board:
        df["board"] = df["board"].astype(str).fillna("")

    for c in ["likes", "views", "comments_num"]:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)

    df["id"] = pd.to_numeric(df["id"], errors="coerce")
    backup_id = df["url"].apply(extract_id_from_url)
    df["post_key"] = df["id"].fillna(backup_id).astype("Int64")

    dt = pd.to_datetime(df["date"], errors="coerce", utc=True)
    dt = dt.dt.tz_convert("Asia/Seoul").dt.tz_localize(None)
    df["date"] = dt

    print(f"  ✓ {len(df):,}행 로드 | date dtype: {df['date'].dtype}")
    return df


START_DT = pd.Timestamp(2024, 9, 1)
END_DT   = pd.Timestamp(2025, 8, 31, 23, 59, 59)


def filter_date_and_dedup(df: pd.DataFrame, name: str) -> pd.DataFrame:
    before = len(df)
    mask = (df["date"] >= START_DT) & (df["date"] <= END_DT)
    df = df.loc[mask].copy()
    after_date = len(df)

    if "post_key" in df.columns:
        df = df.sort_values("date").drop_duplicates(subset=["post_key"], keep="first")
    else:
        df = df.sort_values("date").drop_duplicates(subset=["url"], keep="first")

    after_dedup = len(df)
    print(f"[{name}] {before:,} → 날짜 {after_date:,} → 중복제거 {after_dedup:,}")
    return df


def ensure_post_key(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if "post_key" not in df.columns:
        df["id"] = pd.to_numeric(df["id"], errors="coerce")
        backup = df["url"].apply(extract_id_from_url)
        df["post_key"] = df["id"].fillna(backup).astype("Int64")
    return df


print("✅ 유틸 함수 정의 완료")

---
## 4. CSV 데이터 로드

In [ ]:
# 일반 게시판 로드
ik2_df  = load_csv(DATA_DIR / "익게2_full_details.csv")
free_df = load_csv(DATA_DIR / "자유게시판_full_details.csv")
love_df = load_csv(DATA_DIR / "연애상담소_full_details.csv")
ik1_df  = load_csv(DATA_DIR / "익게1_full_details.csv")

# 인기글 로드 (board 포함)
hot_df  = load_csv(DATA_DIR / "인기글_full_details.csv", has_board=True)

print("\n✅ 모든 CSV 로드 완료")

---
## 5. 날짜 필터링 + 중복 제거 + 게시판 정제 (기존 Phase 0 동일)

In [ ]:
print("📊 날짜 필터링 및 중복 제거 진행 중...\n")

ik2_df  = filter_date_and_dedup(ik2_df,  "익게2")
free_df = filter_date_and_dedup(free_df, "자유게시판")
love_df = filter_date_and_dedup(love_df, "연애상담소")
ik1_df  = filter_date_and_dedup(ik1_df,  "익게1")
hot_df  = filter_date_and_dedup(hot_df,  "인기글(raw)")

# 게시판명 정제
def clean_board_name(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = str(s).strip().strip("|")
    s = re.sub(r"[\[\]\(\)]", "", s)
    return s.strip()

hot_df["board"] = hot_df["board"].astype(str).apply(clean_board_name)

ALLOWED_BOARDS = ["자유게시판", "연애상담소", "익게1", "익게2"]
hot_df = hot_df[hot_df["board"].isin(ALLOWED_BOARDS)].copy()

# post_key 보강
ik2_df  = ensure_post_key(ik2_df)
free_df = ensure_post_key(free_df)
love_df = ensure_post_key(love_df)
ik1_df  = ensure_post_key(ik1_df)
hot_df  = ensure_post_key(hot_df)

print(f"\n📊 인기글 게시판 분포:")
print(hot_df["board"].value_counts())
print(f"\n✅ 기본 전처리 완료: {len(hot_df):,}건")

---
## 6. 🆕 노이즈 게시글 제거

학습 데이터 품질 향상을 위해 5가지 규칙 기반 필터링을 적용합니다.

| 필터 | 기준 | 근거 |
|------|------|------|
| 빈 콘텐츠 | contents가 NaN/빈문자열/공백만 | 본문 없는 글은 학습 불가 |
| 초단문 | 본문 길이 < 10자 | 단순 반응 제거 |
| nan 문자열 | title/contents에 'nan' 포함 | 크롤링 오류 |
| 과도한 URL | URL이 3개 이상 | 광고/스팸 |
| 중복 제목 | 동일 제목 3회 이상 | 봇/도배 |

In [ ]:
def remove_noise_posts(df: pd.DataFrame) -> tuple:
    """
    5가지 규칙 기반 노이즈 필터링.
    Returns:
        (정제된 DataFrame, 제거 통계 dict, 제거 마스크)
    """
    report = {"before": len(df), "filters": {}}
    mask_keep = pd.Series(True, index=df.index)

    # ── 필터 1: 빈 콘텐츠 ─────────────────────────────────
    contents_clean = df["contents"].fillna("").str.strip()
    f1 = (contents_clean == "") | (contents_clean == "nan")
    report["filters"]["빈_콘텐츠"] = int(f1.sum())
    mask_keep &= ~f1

    # ── 필터 2: 초단문 (10자 미만) ─────────────────────────
    f2 = contents_clean.str.len() < 10
    # 필터 1에서 이미 잡힌 것은 중복 카운트 방지
    f2_new = f2 & ~f1
    report["filters"]["초단문_10자미만"] = int(f2_new.sum())
    mask_keep &= ~f2

    # ── 필터 3: nan 문자열 포함 ────────────────────────────
    title_lower = df["title"].fillna("").str.strip().str.lower()
    contents_lower = contents_clean.str.lower()
    # title이 정확히 'nan'이거나, contents가 정확히 'nan'인 경우
    f3 = (title_lower == "nan") | (contents_lower == "nan")
    report["filters"]["nan_문자열"] = int((f3 & ~f1 & ~f2).sum())
    mask_keep &= ~f3

    # ── 필터 4: 과도한 URL (3개 이상) ──────────────────────
    url_pattern = re.compile(r'https?://\S+')
    url_counts = contents_clean.apply(lambda x: len(url_pattern.findall(x)))
    f4 = url_counts >= 3
    f4_new = f4 & mask_keep
    report["filters"]["과도한_URL"] = int(f4_new.sum())
    mask_keep &= ~f4

    # ── 필터 5: 중복 제목 (동일 제목 3회 이상) ──────────────
    title_clean = df["title"].fillna("").str.strip()
    title_counts = title_clean.map(title_clean.value_counts())
    f5 = (title_counts >= 3) & (title_clean != "") & (title_clean.str.lower() != "nan")
    f5_new = f5 & mask_keep
    report["filters"]["중복_제목"] = int(f5_new.sum())
    mask_keep &= ~f5

    # ── 결과 ──────────────────────────────────────────────
    removed_mask = ~mask_keep
    df_clean = df[mask_keep].copy().reset_index(drop=True)
    report["after"] = len(df_clean)
    report["total_removed"] = report["before"] - report["after"]

    return df_clean, report, removed_mask


print("✅ 노이즈 제거 함수 정의 완료")

In [ ]:
print("🧹 노이즈 게시글 제거 시작...\n")

hot_clean_v2, noise_report, removed_mask = remove_noise_posts(hot_df)

# ── 리포트 출력 ──────────────────────────────────────────
print("=" * 60)
print("노이즈 제거 리포트".center(60))
print("=" * 60)
print(f"  전처리 전: {noise_report['before']:,}건")
print(f"  전처리 후: {noise_report['after']:,}건")
print(f"  총 제거:   {noise_report['total_removed']:,}건")
print("─" * 60)
print("  필터별 제거 분포:")
for filter_name, count in noise_report["filters"].items():
    print(f"    {filter_name}: {count:,}건")
print("=" * 60)

# ── 제거된 게시글 샘플 확인 (원래 hot_df에서 추출) ──────────
removed_sample = hot_df[removed_mask][["title", "contents", "board"]].head(5)
print("\n🔍 제거된 게시글 샘플:")
for i, (_, row) in enumerate(removed_sample.iterrows(), 1):
    title = str(row["title"])[:50]
    contents = str(row["contents"])[:80]
    print(f"  [{i}] [{row['board']}] {title}")
    print(f"      내용: {contents}...")

---
## 7. 인기글 원글 제거 + 저장

In [ ]:
# ── 인기글 원글 제거 (토픽 모델링용 게시판 데이터) ──────────
hot_keys = set(hot_clean_v2["post_key"].dropna().astype("Int64"))
print(f"📌 인기글 고유 post_key 수: {len(hot_keys):,}")


def mask_hot(df: pd.DataFrame, name: str) -> pd.DataFrame:
    before = len(df)
    out = df[~df["post_key"].isin(hot_keys)].copy()
    removed = before - len(out)
    print(f"[{name}] {before:,} → {len(out):,} (인기글 제거 {removed:,}개)")
    return out


ik2_for_topic  = mask_hot(ik2_df,  "익게2")
free_for_topic = mask_hot(free_df, "자유게시판")
love_for_topic = mask_hot(love_df, "연애상담소")
ik1_for_topic  = mask_hot(ik1_df,  "익게1")

print("\n✅ 인기글 원글 제거 완료")

In [ ]:
# ── 전처리 결과 저장 ──────────────────────────────────────
print("💾 전처리 데이터 저장 중...")

# 토픽 모델링용 (기존과 동일 경로)
ik2_for_topic.to_parquet(OUTPUT_DIR / "익게2_for_topic.parquet", index=False)
free_for_topic.to_parquet(OUTPUT_DIR / "자유게시판_for_topic.parquet", index=False)
love_for_topic.to_parquet(OUTPUT_DIR / "연애상담소_for_topic.parquet", index=False)
ik1_for_topic.to_parquet(OUTPUT_DIR / "익게1_for_topic.parquet", index=False)

# 🆕 노이즈 제거된 인기글 (v2)
hot_clean_v2.to_parquet(OUTPUT_DIR / "인기글_clean_v2.parquet", index=False)

# 🆕 필터링 리포트 저장
with open(OUTPUT_DIR / "noise_removal_report.json", "w", encoding="utf-8") as f:
    json.dump(noise_report, f, ensure_ascii=False, indent=2)

print("  ✓ 익게2_for_topic.parquet")
print("  ✓ 자유게시판_for_topic.parquet")
print("  ✓ 연애상담소_for_topic.parquet")
print("  ✓ 익게1_for_topic.parquet")
print("  ✓ 인기글_clean_v2.parquet (🆕 노이즈 제거)")
print("  ✓ noise_removal_report.json (🆕 필터링 리포트)")
print(f"\n✅ 모든 파일 저장 완료 → {OUTPUT_DIR}")

---
## 8. 전처리 요약

In [ ]:
summary_rows = [
    {"게시판": "익게2",     "전처리 후": f"{len(ik2_for_topic):,}"},
    {"게시판": "자유게시판", "전처리 후": f"{len(free_for_topic):,}"},
    {"게시판": "연애상담소", "전처리 후": f"{len(love_for_topic):,}"},
    {"게시판": "익게1",     "전처리 후": f"{len(ik1_for_topic):,}"},
    {"게시판": "인기글(v2)", "전처리 후": f"{len(hot_clean_v2):,}"},
]

summary_df = pd.DataFrame(summary_rows)
print("\n" + "=" * 60)
print("Phase 0v2 완료 - 전처리 + 노이즈 제거 요약".center(60))
print("=" * 60)
print(summary_df.to_string(index=False))
print("=" * 60)
print(f"\n🧹 노이즈 제거: {noise_report['before']:,} → {noise_report['after']:,} ({noise_report['total_removed']:,}건 제거)")
print("\n🎉 Phase 0v2 완료!")
print("다음 단계: Phase 1(기존 토픽 모델링) → phase2_dataset_v2.ipynb")